In [3]:
#Directory
dir = "/content/drive/MyDrive/CARD_MPS/FAES/FAES_BIOF509/Final_Project"
# Setting random state for consistency
np.random.seed(123)
df = pd.read_csv(f"{dir}/dementia_patients_health_data.csv")

/usr/local/lib/python3.12/dist-packages/gdown/parse_url.py:48: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=117X6i53dKiO7w6kuA1g1TpdTlv1173h_dPlJt5cNNMU
  warnings.warn(
Downloading...
From: https://docs.google.com/spreadsheets/d/117X6i53dKiO7w6kuA1g1TpdTlv1173h_dPlJt5cNNMU/edit?usp=sharing
To: /content/edit?usp=sharing
252kB [00:00, 1.31MB/s]


In [ ]:
img_dim = 256

In [ ]:

from tensorflow.keras.models import Sequential
from keras.layers import Dense, Flatten
model = Sequential([
    Input(shape = (img_dim, img_dim, 3)),
    Conv2D(8, (3,3), activation='relu', padding='same'),
    Conv2D(8, (3,3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    Conv2D(16, (3,3), activation='relu', padding='same'),
    Conv2D(16, (3,3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    Conv2D(32, (3,3), activation='relu', padding='same'),
    Conv2D(32, (3,3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    Flatten(),
    Dense(128, activation = 'relu'),
    Dense(4, activation = 'softmax')
])
